In [2]:
import os
import cdsapi

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotnine as p9
import seaborn as sns

In [3]:
#!pip install sdcpy
from tqdm.auto import tqdm
from collections import defaultdict
from sdcpy.scale_dependent_correlation import SDCAnalysis
import warnings

c:\Users\brand\anaconda3\envs\tfm_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html


In [4]:
world = pd.read_pickle('../data/world_shape.pkl')

In [5]:
country_to_iso = world[['country', 'iso3']].drop_duplicates()

In [8]:
covid_df = pd.read_csv('../data/covid/covid19_world.csv')

In [9]:
dens = pd.read_csv('../data/processed_05_grid.csv')

In [10]:
country_groups = pd.read_csv('../data/country_groups.csv')

In [11]:
lon_lat_country = pd.read_csv('../data/coords_region.csv').merge(country_to_iso)

In [12]:
lon_lat_group = lon_lat_country.merge(country_groups)

In [13]:
processed_path = "../data/climate/processed/"
relevant_files = [f for f in os.listdir(processed_path) if (f[:9] >= '2021_12_0') & (f[:9] <= '2022_12_2')]
climate_df = pd.concat([pd.read_pickle(f'{processed_path}{f}') for f in relevant_files])

In [17]:
f = (world
     .merge(country_groups)
     .pipe(lambda dd: p9.ggplot(dd)
       + p9.geom_map(data=world, fill='gray')
       + p9.geom_map(p9.aes(fill='group'))
       + p9.ylim(-55, None)
       + p9.labs(fill='')
       + p9.theme_void()
       + p9.theme(figure_size=(9, 4), dpi=200, legend_key_height=15)
      )
)
f.save("../results/world_plot.png")

c:\Users\brand\anaconda3\envs\tfm_env\Lib\site-packages\plotnine\ggplot.py:587: PlotnineWarning: Saving 9 x 4 in image.
c:\Users\brand\anaconda3\envs\tfm_env\Lib\site-packages\plotnine\ggplot.py:588: PlotnineWarning: Filename: ../results/world_plot.png


In [13]:
weighted_cells = (dens.merge(lon_lat_group)
    .groupby('group', as_index=False)
    .apply(lambda gdd: gdd.assign( relative_weight=lambda gdd: gdd.density / gdd.density.sum()))
    .reset_index(drop=True)
)

C:\Users\brand\AppData\Local\Temp\ipykernel_23900\1766496956.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.


In [14]:
weighted_climate_df = (
        weighted_cells.dropna()
        .merge(climate_df, on=["latitude", "longitude"])
        .assign(temperature=lambda dd: dd.temperature * dd.relative_weight,
                absolute_humidity=lambda dd: dd.absolute_humidity * dd.relative_weight)
        .groupby(['group', "date"], as_index=False)
        [["temperature", "absolute_humidity"]]
        .agg({"temperature": "sum", "absolute_humidity": "sum"})
)

In [15]:
groups_covid_ts = \
(covid_df
 .merge(country_groups)
 .assign(date=lambda dd: pd.to_datetime(dd.date))
 .groupby('group')
 .apply(lambda dd: 
        dd.set_index('date')
        .resample('D')
        .sum()
        ['new_cases']
        #.rolling(center=True, window=7)
        #.mean()
        .loc['2021-12-01': '2022-12-31'])
)

C:\Users\brand\AppData\Local\Temp\ipykernel_23900\1085163371.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.


In [17]:
groups_weather_ts = \
(weighted_climate_df
 .assign(date=lambda dd: pd.to_datetime(dd.date))
 .query('"2021-12-01" <= date <= "2022-12-31"')
 .set_index(['group', 'date'])
)

In [21]:
CV_LABELS = {'absolute_humidity': 'Absolute Humidity [g/m³]',
             'temperature': 'Temperature [°C]',
             'relative_humidity': 'Relative Humidity [%]',
             'total_precipitation': 'Total Precipitation [mm]'}

## SDC Analysis for groups of contiguous countries

In [23]:
import openpyxl

w = 105
out_dir = '../results'
for group in tqdm(country_groups.group.unique(), desc='Processing Groups', leave=False):
    for weather_variable in ['temperature', 'absolute_humidity']:
        covid_ts = groups_covid_ts.loc[group]
        weather_ts = groups_weather_ts.loc[group][weather_variable]
        sdc = SDCAnalysis(ts1=weather_ts,
                          ts2=covid_ts,
                          fragment_size=w,
                          method='spearman',
                          max_lag=0,
                          min_lag=-21)
        sdc.to_excel(f"{out_dir}/tables/SDC_country_groups/sdc_{w}_{group}_{weather_variable}.xlsx")
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            sdc.combi_plot(dpi=120,
                           figsize=(7, 7),
                           title=f"SDC with s={w}: \nCOVID19 daily confirmed cases in" +
                                 f" {group} ~ {weather_variable}",
                           xlabel=CV_LABELS[weather_variable],
                           ylabel=f"Confirmed COVID19 cases in {group}",
                           alpha=0.05,
                           max_lag=0,
                           min_lag=-21,
                           max_r=1,
                           wspace=.35,
                           hspace=.35)
        plt.savefig(f"{out_dir}/figures/SDC_country_groups/sdc_{w}_{group}_{weather_variable}.png")
        plt.savefig(f"{out_dir}/figures/SDC_country_groups/sdc_{w}_{group}_{weather_variable}.pdf")

## SDC Analysis for European countries

In [79]:
eur_countries = lon_lat_country[lon_lat_country['country'].isin(['Germany', 'Spain', 'France', 'United Kingdom', 'Italy'])]

In [124]:
eur_covid_ts = (
    covid_df
    .query("country in ['Germany', 'Spain', 'France', 'United Kingdom', 'Italy']")
    .assign(date=lambda dd: pd.to_datetime(dd['date']))
    .set_index('date')
    .sort_index()
    [['country','new_cases']]
    .loc['2021-12-01':'2022-12-31']
)

In [139]:
eur_weighted_cells = (dens.merge(lon_lat_country)
    .groupby('country', as_index=False)
    .apply(lambda gdd: gdd.assign( relative_weight=lambda gdd: gdd.density / gdd.density.sum()))
    .reset_index(drop=True)
)

C:\Users\brand\AppData\Local\Temp\ipykernel_23900\3096511853.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.


In [160]:
eur_weighted_climate_df = (
        eur_weighted_cells.dropna()
        .merge(climate_df, on=["latitude", "longitude"])
        .assign(temperature=lambda dd: dd.temperature * dd.relative_weight,
                absolute_humidity=lambda dd: dd.absolute_humidity * dd.relative_weight)
        .groupby(["date","country_x"], as_index=False)
        [["temperature", "absolute_humidity"]]
        .agg({"temperature": "sum", "absolute_humidity": "sum"})
)

In [164]:
eur_weather_ts = (
    eur_weighted_climate_df
    .query("country_x in ['Germany', 'Spain', 'France', 'United Kingdom', 'Italy']")
    .assign(date=lambda dd: pd.to_datetime(dd['date']))
    .query('"2021-12-01" <= date <= "2022-12-31"')
    .set_index('date')
)

In [167]:

w = 105
out_dir = '../results'
for country in tqdm(['Germany', 'Spain', 'France', 'United Kingdom', 'Italy'], desc='Processing Countries', leave=False):
    for weather_variable in ['temperature', 'absolute_humidity']:
        country_covid_ts = eur_covid_ts[eur_covid_ts['country'] == country]['new_cases']
        country_weather_ts = eur_weather_ts[eur_weather_ts['country_x'] == country][weather_variable]
        sdc = SDCAnalysis(ts1=country_weather_ts,
                          ts2=country_covid_ts,
                          fragment_size=w,
                          method='spearman',
                          max_lag=0,
                          min_lag=-21)
        sdc.to_excel(f"{out_dir}/tables/SDC_eur_countries/sdc_{w}_{country}_{weather_variable}.xlsx")
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            sdc.combi_plot(dpi=120,
                           figsize=(7, 7),
                           title=f"SDC with s={w}: \nCOVID19 daily confirmed cases in" +
                                 f" {country} ~ {weather_variable}",
                           xlabel=CV_LABELS[weather_variable],
                           ylabel=f"Confirmed COVID19 cases in {country}",
                           alpha=0.05,
                           max_lag=0,
                           min_lag=-21,
                           max_r=1,
                           wspace=.35,
                           hspace=.35)
        plt.savefig(f"{out_dir}/figures/SDC_eur_countries/sdc_{w}_{country}_{weather_variable}.png")
        plt.savefig(f"{out_dir}/figures/SDC_eur_countries/sdc_{w}_{country}_{weather_variable}.pdf")
        plt.close()